In [283]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
import os

from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

In [127]:
z_df = pd.read_csv("/home/justin/Downloads/emissions/zhvi.csv")
z_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30
0,91940,0,77449,zip,TX,TX,NaN,"Houston-The Woodlands-Sugar Land, TX",Harris County,106672.0,...,251930.0,255861.0,260506.0,266174.0,272442.0,278528.0,283555.0,286948.0,288863.0,289504.0
1,91982,1,77494,zip,TX,TX,NaN,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,223357.0,...,439209.0,445272.0,454755.0,466373.0,480063.0,491447.0,498934.0,501198.0,499334.0,499877.0
2,93144,2,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,90482.0,...,174265.0,176277.0,178252.0,180407.0,183773.0,187384.0,190834.0,193074.0,194779.0,196166.0
3,62080,3,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,187749.0,...,571191.0,569363.0,569160.0,571091.0,572605.0,575407.0,577640.0,579764.0,580707.0,583489.0
4,62093,4,11385,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,275753.0,...,744928.0,748113.0,751397.0,757101.0,759701.0,765523.0,770799.0,775959.0,778630.0,781713.0


In [218]:
emissions_df = pd.read_csv("/home/justin/Downloads/emissions/ghgp_data_by_year.csv", usecols = [3,4,5,13,14,15,16,17,18,19,20,21,22,23], skiprows = [0,1,2])
emissions_df = emissions_df.fillna(0)
emissions_np = emissions_df.to_numpy()
emissions_df.head()
cities = emissions_df['City'].unique()
cities = [str(c).upper() for c in cities]
print(len(cities))

4899


In [219]:
realty_df = pd.read_csv("/home/justin/Downloads/emissions/realtor-data.csv",usecols=[1,7,8,9,11])
realty_df.dropna(subset=['sold_date'])
realty_df = realty_df[(realty_df.sold_date > '2011-01-01') & (realty_df.sold_date < '2022-01-01')]
realty_df['city'] = realty_df['city'].str.upper()

In [220]:
realty_df.head()


,price,city,state,zip_code,sold_date
684,76900.0,CANOVANAS,Puerto Rico,729.0,2020-02-28
745,76900.0,CANOVANAS,Puerto Rico,729.0,2020-02-28
802,76900.0,CANOVANAS,Puerto Rico,729.0,2020-02-28
829,110000.0,DORADO,Puerto Rico,949.0,2019-06-28
932,76900.0,CANOVANAS,Puerto Rico,729.0,2020-02-28


In [303]:
realty_train = realty_df.sample(frac=0.5,random_state=200)
realty_test = realty_df.drop(realty_train.index)
realty_train = realty_train.reset_index()
cities1 = realty_train["city"].unique()
cities1 = [c.upper() for c in cities1]

cities2 = realty_test["city"].unique()
cities2 = [c.upper() for c in cities2]
print(len(cities1))

1657


In [304]:
set_dif = set(cities).symmetric_difference(set(cities1))

set_dif2 = set(cities).symmetric_difference(set(cities2))

print(len(set(cities).symmetric_difference(set(cities1))))


4641


In [305]:
emissions_df['City'] = emissions_df['City'].str.upper()
emissions_train = emissions_df[~emissions_df["City"].isin(list(set_dif))]
realty_train = realty_train[~realty_train["city"].isin(list(set_dif))]

realty_test = realty_test[~realty_test["city"].isin(list(set_dif2))]
emissions_test = emissions_df[~emissions_df["City"].isin(list(set_dif2))]

print(emissions_train.shape)
print(realty_train.shape)

(1606, 14)
(64201, 6)


In [297]:
#TODO: convert cities to CBSA in order to bring in air quality data from EPA

In [298]:
#realty_test = realty_test.reset_index()
#realty_test.drop("index", axis=1)
realty_test.shape

(107083, 5)

In [309]:
realty_train_labels_grp = realty_train.groupby(["city"])
realty_train_labels_np = realty_train_labels_grp['price'].agg(np.mean).to_numpy()
print(realty_train_labels_np.shape)

emissions_grp = emissions_train.groupby(["City"])
emissions_zip = emissions_grp['2021 Total reported direct emissions'].agg(np.sum).to_numpy()
print(emissions_zip.shape)

realty_train_labels_grp_test = realty_test.groupby(["city"])
realty_train_labels_test = realty_train_labels_grp_test['price'].agg(np.mean).to_numpy()

emissions_grp_test = emissions_test.groupby(["City"])
emissions_zip_test = emissions_grp_test['2021 Total reported direct emissions'].agg(np.sum).to_numpy()


(485,)
(485,)


In [316]:
prepro_x = MinMaxScaler()
tez = emissions_zip.reshape(-1,1)
prepro_x.fit(tez)
emissions_zip = prepro_x.transform(tez)

rtln = realty_train_labels_np.reshape(-1,1)
prepro_y = MinMaxScaler()
prepro_y.fit(rtln)
realty_train_labels_np = prepro_y.transform(rtln)

prepro_z = MinMaxScaler()
tez = emissions_zip_test.reshape(-1,1)
prepro_z.fit(tez)
emissions_zip_test = prepro_x.transform(tez)

rtln = realty_train_labels_test.reshape(-1,1)
prepro_y = MinMaxScaler()
prepro_y.fit(rtln)
realty_train_labels_test = prepro_y.transform(rtln)


In [324]:

def build_model():
    model = keras.Sequential([
        Dense(64, activation=tf.nn.relu, input_dim=1),
        Dense(64, activation=tf.nn.relu),
        Dense(1)
    ])

    model.compile(optimizer='adam', 
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model



In [330]:
model = build_model()

history = model.fit(emissions_zip, realty_train_labels_np, epochs=1000, batch_size=256, verbose=0, validation_split = 0.1)


print()
results = model.evaluate(emissions_zip_test, realty_train_labels_test)
print(model.metrics_names)
print(results)

KeyboardInterrupt: 